# Fase 2: Extracción de Detalles de Películas con Selenium

WEBS
- IMDB: https://www.imdb.com/
- Rotten Tomatoes: https://www.rottentomatoes.com/

- Puntuacion de IMDB (en caso de que la tenga). ✅

- Puntuacion de Rotten Tomatoes (Tomatometer).

- Dirección (director/a o directore/as de cada película). ✅

- Gionistas (de cada película). ✅

- Argumento.

- Duración (en minutos).

- Nombre de la película

In [4]:
#Importamos las librerias necesarias
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
from time import sleep

#importamos librerias de selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

In [6]:
mini_lista = ['tt10579986',
 'tt10597228',
 'tt11097384',
 'tt11152168',
 'tt11237950']

In [7]:
# creamos listas para almacenar la información obtetina de las webs
lista_punt_IMDB = []
lista_punt_tomato = []
lista_dire = []
lista_guionistas = []
lista_argumento = []
lista_duracion = []

In [8]:
# Inicializa el navegador Chrome
driver = webdriver.Chrome()

# Navega a la página web "http://www.imdb.com"
driver.get("http://www.imdb.com")

# Maximizar la ventana del navegador
driver.maximize_window()

# Acepta las cookies haciendo clic en el elemento con el selector CSS
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

sleep(5)

#Iniciamos loop for para iterar en la lista de todas las películas
for peli in mini_lista:

    # Introducimos el número de ID de la peli en el buscador de la web
    driver.find_element('css selector', '#suggestion-search').send_keys(peli, Keys.ENTER)
    sleep(2)


    #Buscamos la información de las puntaciones de cada película
    #Empezamos un try/except loop per si hay películas que no tienen puntuación. En este caso appendearemos NULL en la lista
    try:
        lista_punt_IMDB.append(float(driver.find_element('css selector', '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-3.frxYSZ > div.sc-3a4309f8-0.bjXIAP.sc-b7c53eda-1.iIQkEw > div > div:nth-child(1) > a > span > div > div.sc-bde20123-0.dLwiNw > div.sc-bde20123-2.cdQqzc > span.sc-bde20123-1.cMEQkK').text))
    except:
        lista_punt_IMDB.append('NULL')


    #Buscamos la información de los directorters de cada película  
    # Definimos los valores iniciales de nuestras variables para iniciar un loop while que serán:
    # un contador i y una lista vacía donde guardar todos los directores de ésta película
    i=0
    directores=[]
    #Iniciamos el loop while
    while True:  
        i+=1
        # Iniciamos try/expect loop para que cuando no tengamos más directores dé error y salganmos del loop while
        try:
            directores.append(driver.find_element('xpath', f'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/div[2]/div/ul/li[1]/div/ul/li[{i}]/a').text)
        except:
            break
    #Appendeamos la lista de directores de esta película en la lista general de directores
    lista_dire.append(directores)


    #Buscamos la información de los guionistas de cada película  
    # Definimos los valores iniciales de nuestras variables para iniciar un loop while que serán:
    # un contador i y una lista vacía donde guardar todos los guionistas de ésta película
    i=0
    guionistas=[]
    #Iniciamos el loop while
    while True:  
        i+=1
        # Iniciamos try/expect loop para que cuando no tengamos más guionistas dé error y salganmos del loop while
        try:
            guionistas.append(driver.find_element('xpath', f'/html/body/div[2]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/div[2]/div/ul/li[2]/div/ul/li[{i}]/a').text)
        except:
            break
    #Appendeamos la lista de guinistas de esta película en la lista general de guionistas
    lista_guionistas.append(guionistas)


    #Buscamos el argumento de cada película  
    #Empezamos iniciando un while loop para ir haciendo scroll down hasta encontrar la parte de la web donde esta el argumento
    #Definimos el valor de las variables que usaremos en el while loop
    i=0
    #Iniciamos el loop while
    while True:
        #Incluimos un máximo de scroll down por si no alguna película no tiene argumento
        if i<5000:
            try:
                #Hacemos scroll down
                driver.execute_script(f'window.scrollTo(0,{i})')
                sleep(3)
                #Appendeamos el argumento a la lista de argumentos
                lista_argumento.append(driver.find_element('xpath','/html/body/div[2]/main/div/section[1]/div/section/div/div[1]/section[3]/div[2]/div[1]/div/div/div').text)
                break
            except:
                #Si no se ha encontrado el argumento añadimos 500 pixeles más para hacer scroll down
                i+=500
        else:
            lista_argumento.append('NULL')
            break
    #Hacemos scroll up para volver al inicio de la página y poder seguir con la búsqueda de info
    driver.execute_script(f'window.scrollTo({i},0)')

    


driver.close()   

## TESTEANDO 👇🏻

In [8]:
# Inicializa el navegador Chrome
driver = webdriver.Chrome()

# Navega a la página web "http://www.imdb.com"
driver.get("http://www.imdb.com")

# Maximizar la ventana del navegador
driver.maximize_window()

# Acepta las cookies haciendo clic en el elemento con el selector CSS
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

sleep(2)


driver.find_element('css selector', '#suggestion-search').send_keys('tt11152168', Keys.ENTER)
sleep(2)

# i=0
# while True:
    
#     try:
#         driver.execute_script(f'window.scrollTo(0,{i})')
#         sleep(2)
text= driver.find_element('div','class:{ipc-html-content-inner-div}')
#         break
#     except:
#         i+=500

# driver.execute_script(f'window.scrollTo({i},0)')

# driver.close()
        
        




# driver.find_element('css selector', '#__next > button').click()
# sleep(2)

# driver.close()   

InvalidArgumentException: Message: invalid argument: invalid locator
  (Session info: chrome=124.0.6367.119)
Stacktrace:
0   chromedriver                        0x000000010f218a18 chromedriver + 5216792
1   chromedriver                        0x000000010f210653 chromedriver + 5183059
2   chromedriver                        0x000000010ed82527 chromedriver + 406823
3   chromedriver                        0x000000010edcd8a1 chromedriver + 714913
4   chromedriver                        0x000000010edce181 chromedriver + 717185
5   chromedriver                        0x000000010ee121d4 chromedriver + 995796
6   chromedriver                        0x000000010edf072d chromedriver + 857901
7   chromedriver                        0x000000010ee0f6bf chromedriver + 984767
8   chromedriver                        0x000000010edf04a3 chromedriver + 857251
9   chromedriver                        0x000000010edc0fe3 chromedriver + 663523
10  chromedriver                        0x000000010edc192e chromedr

In [ ]:
/html/body/div[2]/main/div/section[1]/div/section/div/div[1]/section[5]/div[2]/div[1]/div/div/div
/html/body/div[2]/main/div/section[1]/div/section/div/div[1]/section[3]/div[2]/div[1]/div/div/div

In [12]:
text

"Newly engaged lovers, Jason and Brandon return to Los Angeles after two years in New York after Brandon completes his contract as a dancer with the New York Ballet Company. While Jason goes back to work as a piano accompanist at the local ballet studio, Brandon transitions into teaching ballet students. Jason meets a stunningly good-looking dancer Devon with whom he has a strong connection who tries to convince him to help him with his career through the rumored magical healing power of Jason's piano music. Brandon's former partner, Adam, who was violent and abusive when they were together, is back in the picture and making a play for Brandon. Even though Adam has gotten his life together, he is still desperately in love with Brandon. As Jason and Brandon settle into their routine of work and daily life, memories of a tragic family car accident still haunt Jason. Jason's wish of having his children, Max and Isabella, back seems like it may become attainable through his ability to heal

In [10]:
lista_argumento

["A soul allegedly with a 20-year-old girl as her daughter, whom she wants to take with her and the elder sister's wants to stop the soul from doing so.",
 "This is a story about finding the courage to be one's self.",
 'NULL',
 'NULL',
 "Newly engaged lovers, Jason and Brandon return to Los Angeles after two years in New York after Brandon completes his contract as a dancer with the New York Ballet Company. While Jason goes back to work as a piano accompanist at the local ballet studio, Brandon transitions into teaching ballet students. Jason meets a stunningly good-looking dancer Devon with whom he has a strong connection who tries to convince him to help him with his career through the rumored magical healing power of Jason's piano music. Brandon's former partner, Adam, who was violent and abusive when they were together, is back in the picture and making a play for Brandon. Even though Adam has gotten his life together, he is still desperately in love with Brandon. As Jason and Bra